In [ ]:
# run if datasketch not already installed
# %pip install datasketch

Python interpreter will be restarted.
Python interpreter will be restarted.


<h1 style="text-align: center;">
Duplicate Detection
</h1>

Assume I was reading news items in the HuffPost website (https://www.huffpost.com/) and last clicked on a news article about gun control in the USA. The next day, I return to this website and it decides to prioritize new items for me based on the last news item I read. Use the Minhash/LSH algorithm to find URL link, headline, category, and short description of the 5 most similar new item (based on the "short_description" field: "Police reportedly used stun guns on at least two parents who arrived for their kids after reports of an armed man on a school campus." )

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import FloatType

from datasketch import MinHash

spark = SparkSession.builder.appName("dh3382-midterm-duplicate-detection").getOrCreate()

HUFF_PATH = '/FileStore/tables/Huffpost.json'

# description string to compare with all entries in Huffpost json
DESC_COMP = 'Police reportedly used stun guns on at least two parents who arrived for their kids after reports of an armed man on a school campus.'

# split DESC_COMP into array of strings for jaccard similarity processing
DESC_COMP_STR_ARR = DESC_COMP.split()

I first define a UDF using datasketch's jaccard similarity estimation function. I decided to use the built in PySpark split function to convert the short descriptions into string arrays trusting that it would be more efficient than a custom implementation, so the jaccard UDF takes in column values that have already been converted into string arrays. Following datasketch documentation, I then create two MinHash objects, one for each string array to be compared, and encode each using the MinHash update function. I use a constant string variable, DESC_COMP_STR_ARR to compare each value to the description specified in the prompt (alternatively, I could have added a column to the huff_df where all values were DESC_COMP_STR_ARR and passed that value in to the jaccard UDF. This would have made for a more general usage version of my jaccard udf, but otherwise similar results).

In [ ]:
# jaccard UDF, compare with DESC_COMP_STR_ARR to avoid having to create extra column with const values
def jaccard(str_arr):
    m1, m2 = MinHash(), MinHash()
    for s in str_arr:
        m1.update(s.encode('utf8') )
    for s in DESC_COMP_STR_ARR:
        m2.update(s.encode('utf8') )
    
    return m1.jaccard(m2)

jaccard_udf = udf(lambda str_arr: jaccard(str_arr), FloatType() )

I then read in the json file, filter out any columns not needed for the result, and convert the short_description column from strings to arrays of strings to I can pass it in to the jaccard UDF. I initially filtered out all null entries, as well, but found none so I removed the function. I create a jaccard_similarity column using the jaccard UDF which compares the short_description in each row to the string specified in the prompt and produces a jaccard similarity float. I persist this result because Databricks was throttling my account, and with a much larger dataset this would also allow for faster actions on the dataframe with the jaccard_similarity column. I filter out all results with a jaccard similarity less than 0.1 to heuristically optimize sorting, then sort the table in descending order of jaccard similarity and show the top 5 most similar articles.

In [ ]:
# read in json file
huff_raw = spark.read.json(HUFF_PATH)

# filter out unnecessary columns 'authors' and 'date', initially filtered for nulls as well but none were present in the dataset
huff_df = huff_raw.select(col('link'), col('headline'), col('category'), col('short_description') )

# convert 'short_description' values from string to array of strings for MinHash processing
huff_df = huff_df.withColumn('short_description', split('short_description', ' +') )

# add jaccard similarity column using jaccard UDF
huff_df = huff_df.withColumn('jaccard_similarity', jaccard_udf(col('short_description') ) )

# persist to avoid recalculating jaccard similarity on sort
huff_df.persist()

# filter out low similarity results before sorting to optimize
huff_df = huff_df.filter(col('jaccard_similarity') > 0.1)

# sort on jaccard similarity to find most similar articles
duplicate_detection_res = huff_df.sort('jaccard_similarity', ascending=False)

# show top 5 results
duplicate_detection_res.toPandas().head(5)

,link,headline,category,short_description,jaccard_similarity
0,https://www.huffpost.com/entry/parents-arreste...,Parents Arrested After Attempting To Grab Thei...,U.S. NEWS,"[Police, reportedly, used, stun, guns, on, at,...",1.000000
1,https://www.huffpost.com/entry/south-africa-ba...,At Least 15 Killed In South Africa Bar Shooting,WORLD NEWS,"[Police, say, they, are, investigating, report...",0.210938
2,https://www.huffingtonpost.com/entry/battles-r...,"Battles Rage For Yemen's Aden, Dozens Of Civil...",THE WORLDPOST,"[SANAA,, Yemen, (AP), ―, Shiite, rebels, and, ...",0.210938
3,https://www.huffingtonpost.com/entry/financial...,"Financial Advisors Are Biased, Study Finds",MONEY,"[Based, on, his, research,, Kritzman,, who, te...",0.210938
4,https://www.huffingtonpost.com/entry/donald-tr...,Trump Reportedly Once Bragged About 'First-Rat...,POLITICS,"[The, president,, who, has, been, accused, by,...",0.203125
